In [182]:
import numpy as np 
import pandas as pd
#from pandas_profiling import ProfileReport
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
#from fancyimpute import KNN



In [183]:
churn = pd.read_csv('data/churn.csv')

In [185]:
churn.head(20)

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,6.94,5.0,5.0,1.00,Astapor,2014-05-03,Android,2014-01-12,0.0,0,False,100.0
1,8.06,5.0,5.0,1.00,Astapor,2014-01-26,Android,2014-01-25,0.0,2,True,0.0
2,21.50,4.0,NaN,1.00,Winterfell,2014-05-21,iPhone,2014-01-02,0.0,1,True,100.0
3,9.46,5.0,NaN,2.75,Winterfell,2014-01-10,Android,2014-01-09,100.0,1,False,100.0
4,13.77,5.0,NaN,1.00,Winterfell,2014-05-13,iPhone,2014-01-31,0.0,0,False,100.0
5,14.51,5.0,NaN,1.00,Astapor,2014-04-22,iPhone,2014-01-29,0.0,0,True,100.0
6,4.87,4.7,4.0,1.00,Astapor,2014-04-21,Android,2014-01-18,0.0,0,False,66.7
7,1.78,4.0,5.0,1.00,Winterfell,2014-01-12,iPhone,2014-01-11,0.0,3,False,0.0
8,6.87,5.0,4.7,1.00,Astapor,2014-01-30,Android,2014-01-01,0.0,3,False,66.7
9,2.41,5.0,3.0,1.00,Winterfell,2014-01-29,iPhone,2014-01-28,0.0,1,False,100.0


In [186]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_dist                40000 non-null  float64
 1   avg_rating_by_driver    39838 non-null  float64
 2   avg_rating_of_driver    33472 non-null  float64
 3   avg_surge               40000 non-null  float64
 4   city                    40000 non-null  object 
 5   last_trip_date          40000 non-null  object 
 6   phone                   39681 non-null  object 
 7   signup_date             40000 non-null  object 
 8   surge_pct               40000 non-null  float64
 9   trips_in_first_30_days  40000 non-null  int64  
 10  luxury_car_user         40000 non-null  bool   
 11  weekday_pct             40000 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 3.4+ MB


In [189]:
#Simple mean impute for ratings columns
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imp.fit(churn[['avg_rating_by_driver','avg_rating_of_driver']])
churn_imp = imp.transform(churn[['avg_rating_by_driver','avg_rating_of_driver']])
pd.DataFrame(churn_imp)
churn[['avg_rating_by_driver','avg_rating_of_driver']]=churn_imp
churn

#encode non-null values: https://towardsdatascience.com/preprocessing-encode-and-knn-impute-all-categorical-features-fast-b05f50b4dfaa 
encoder = OrdinalEncoder()
#imputer = KNN()
cat_cols = ['phone']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data


for columns in cat_cols:
    encode(churn[columns])

def churn_(df, column):
    dates = df[column]
    cutoff_date = '2014-06-01'
    array = []
    for date in dates:
        if date > cutoff_date:
            array.append(1)
        else:
            array.append(0)
    df['active'] = array
    return df


churn = churn_(churn, 'last_trip_date')


#KNN impute for phone column
imputer = KNNImputer(n_neighbors=5, weights="uniform")
churn_imp2=imputer.fit_transform(churn[['avg_rating_by_driver','avg_rating_of_driver','phone']])
churn[['avg_rating_by_driver','avg_rating_of_driver','phone']]=churn_imp2


# creating instance of one-hot-encoder 
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(churn[['city']]).toarray())
churn = churn.join(enc_df)
churn = churn.drop(['city'],axis =1)

#create feature with difference between signup and last trip
churn['last_trip_date_d'] = pd.to_datetime(churn['last_trip_date'])
churn['signup_date_d'] = pd.to_datetime(churn['signup_date'])
churn['date_delta'] = churn['last_trip_date_d']-churn['signup_date_d']


In [190]:

churn = churn.rename(columns = {0:'city_0', 1:'city_1',2:'city_2'},errors = 'raise')


In [191]:
churn

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,active,city_0,city_1,city_2,last_trip_date_d,signup_date_d,date_delta
0,6.94,5.0,5.000000,1.00,2014-05-03,0.0,2014-01-12,0.0,0,False,100.0,0,1.0,0.0,0.0,2014-05-03,2014-01-12,111 days
1,8.06,5.0,5.000000,1.00,2014-01-26,0.0,2014-01-25,0.0,2,True,0.0,0,1.0,0.0,0.0,2014-01-26,2014-01-25,1 days
2,21.50,4.0,4.601697,1.00,2014-05-21,1.0,2014-01-02,0.0,1,True,100.0,0,0.0,0.0,1.0,2014-05-21,2014-01-02,139 days
3,9.46,5.0,4.601697,2.75,2014-01-10,0.0,2014-01-09,100.0,1,False,100.0,0,0.0,0.0,1.0,2014-01-10,2014-01-09,1 days
4,13.77,5.0,4.601697,1.00,2014-05-13,1.0,2014-01-31,0.0,0,False,100.0,0,0.0,0.0,1.0,2014-05-13,2014-01-31,102 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,2.06,4.8,4.300000,1.08,2014-04-02,0.0,2014-01-26,9.5,8,False,90.5,0,0.0,0.0,1.0,2014-04-02,2014-01-26,66 days
39996,2.05,5.0,5.000000,1.00,2014-05-09,1.0,2014-01-08,0.0,2,False,85.7,0,0.0,1.0,0.0,2014-05-09,2014-01-08,121 days
39997,3.04,5.0,5.000000,1.00,2014-06-24,0.0,2014-01-04,0.0,3,True,33.3,1,0.0,0.0,1.0,2014-06-24,2014-01-04,171 days
39998,3.49,4.3,3.300000,1.50,2014-02-09,1.0,2014-01-08,40.0,5,False,60.0,0,1.0,0.0,0.0,2014-02-09,2014-01-08,32 days


In [192]:
def date_to_month(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df['month '+ column_name] = df[column_name].dt.month
    return df

date_to_month(churn, 'signup_date')
churn = churn.drop(['signup_date'],axis =1)
date_to_month(churn, 'last_trip_date')
churn = churn.drop(['last_trip_date'], axis = 1)




In [193]:
churn

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,active,city_0,city_1,city_2,last_trip_date_d,signup_date_d,date_delta,month signup_date,month last_trip_date
0,6.94,5.0,5.000000,1.00,0.0,0.0,0,False,100.0,0,1.0,0.0,0.0,2014-05-03,2014-01-12,111 days,1,5
1,8.06,5.0,5.000000,1.00,0.0,0.0,2,True,0.0,0,1.0,0.0,0.0,2014-01-26,2014-01-25,1 days,1,1
2,21.50,4.0,4.601697,1.00,1.0,0.0,1,True,100.0,0,0.0,0.0,1.0,2014-05-21,2014-01-02,139 days,1,5
3,9.46,5.0,4.601697,2.75,0.0,100.0,1,False,100.0,0,0.0,0.0,1.0,2014-01-10,2014-01-09,1 days,1,1
4,13.77,5.0,4.601697,1.00,1.0,0.0,0,False,100.0,0,0.0,0.0,1.0,2014-05-13,2014-01-31,102 days,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,2.06,4.8,4.300000,1.08,0.0,9.5,8,False,90.5,0,0.0,0.0,1.0,2014-04-02,2014-01-26,66 days,1,4
39996,2.05,5.0,5.000000,1.00,1.0,0.0,2,False,85.7,0,0.0,1.0,0.0,2014-05-09,2014-01-08,121 days,1,5
39997,3.04,5.0,5.000000,1.00,0.0,0.0,3,True,33.3,1,0.0,0.0,1.0,2014-06-24,2014-01-04,171 days,1,6
39998,3.49,4.3,3.300000,1.50,1.0,40.0,5,False,60.0,0,1.0,0.0,0.0,2014-02-09,2014-01-08,32 days,1,2
